In [ ]:
# UPLOAD THE DATA AND CHECK THE DURATION OF THE FIRST 1000 POINTS

import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Read in the CSV files and create dataframes, dropping rows with missing values and resetting index
dfs = {}
x_values = {}
y_values = {}
t_values = {}
velocity_values = {}

participant_numbers = [] # Initialize a list to store participant numbers


# Add participants with ADHD to dfs dictionary
i_start_adhd = 1
i_stop_adhd = 29

for i in range(i_start_adhd, i_stop_adhd):
    csv_file = 'data{}.csv'.format(i)
    df_name = 'df{}'.format(i)
    dfs[df_name] = pd.read_csv(csv_file).dropna().reset_index(drop=True)
    x_name = 'x{}'.format(i)
    y_name = 'y{}'.format(i)
    t_name = 't{}'.format(i)
    v_name = 'v{}'.format(i)
    x_values[x_name] = dfs[df_name]['Position_1'].values
    y_values[y_name] = dfs[df_name]['Position_2'].values
    t_values[t_name] = dfs[df_name]['Time'].values
    dx = np.diff(x_values[x_name])
    dy = np.diff(y_values[y_name])
    dt = np.diff(t_values[t_name])
    velocity_values[v_name] = np.sqrt(dx**2 + dy**2) / dt
    
    if len(velocity_values[v_name]) > 0: # Only add participant number if the velocity array is not empty
        participant_numbers.append(str(i)) # Convert the participant number to a string and add it to the list
        
# Add participants without ADHD to dfs dictionary
i_start_no_adhd = 29
i_stop_no_adhd = 51

for i in range(i_start_no_adhd, i_stop_no_adhd):
    csv_file = 'data{}.csv'.format(i)
    df_name = 'df{}'.format(i)
    dfs[df_name] = pd.read_csv(csv_file).dropna().reset_index(drop=True)
    x_name = 'x{}'.format(i)
    y_name = 'y{}'.format(i)
    t_name = 't{}'.format(i)
    v_name = 'v{}'.format(i)
    x_values[x_name] = dfs[df_name]['Position_1'].values
    y_values[y_name] = dfs[df_name]['Position_2'].values
    t_values[t_name] = dfs[df_name]['Time'].values
    dx = np.diff(x_values[x_name])
    dy = np.diff(y_values[y_name])
    dt = np.diff(t_values[t_name])
    velocity_values[v_name] = np.sqrt(dx**2 + dy**2) / dt
    
    if len(velocity_values[v_name]) > 0: # Only add participant number if the velocity array is not empty
        participant_numbers.append(str(i)) # Convert the participant number to a string and add it to the list
        
# Add the code snippet here to calculate and print the duration for the first 1000 points for each participant
durations = {}
for participant in participant_numbers:
    num_points = min(1000, len(t_values['t' + participant]) - 1)
    duration = t_values['t' + participant][num_points] - t_values['t' + participant][0]
    durations[participant] = duration

for participant, duration in durations.items():
    print(f"Participant {participant}: {duration:.2f} milliseconds for the first {num_points} points")
    
# Create the threshold_table dataframe using the entire contents of the original dataframe
df_temp = pd.read_csv("VT.csv")
threshold_table = df_temp.copy()

In [ ]:
# CREATE SCATTER PLOTS FOR THE FIRST 1000 POINTS
# Import additional libraries
import seaborn as sns

# Create a new dictionary to store saccades and fixations data
saccades_fixations = {}

# Iterate through the participant_numbers list
for i in participant_numbers:
    # Retrieve the corresponding threshold value from the threshold_table dataframe
    threshold = threshold_table.loc[threshold_table['Participant'] == int(i), 'Velocity Threshold'].values[0]

    # Identify the fixations and saccades based on the threshold value
    saccades = np.where(velocity_values['v' + i] > threshold)[0]
    fixations = np.where(velocity_values['v' + i] <= threshold)[0]

    # Store the saccades and fixations data in the dictionary
    saccades_fixations['participant' + i] = {'saccades': saccades, 'fixations': fixations}

# Plot the first 1000 points with different colors or markers to distinguish fixations and saccades
for i in participant_numbers:
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.set_title("Participant " + i)

    # Iterate through the first 1000 points
    for j in range(min(1000, len(x_values['x' + i]) - 1)):
        if j in saccades_fixations['participant' + i]['fixations']:
            # Plot fixations
            ax.plot(x_values['x' + i][j], y_values['y' + i][j], 'bo', markersize=5, label='Fixations')
        elif j in saccades_fixations['participant' + i]['saccades']:
            # Plot saccades
            ax.plot(x_values['x' + i][j], y_values['y' + i][j], 'ro', markersize=2, label='Saccades')

    # Create a custom legend to avoid duplicate labels
    custom_legend = [plt.Line2D([0], [0], color='b', marker='o', linestyle='', markersize=5, label='Fixations'),
                     plt.Line2D([0], [0], color='r', marker='o', linestyle='', markersize=2, label='Saccades')]
    ax.legend(handles=custom_legend)
    plt.show()


In [ ]:
# CREATE SCATTER PLOTS FOR THE FIRST 1000 POINTS FOR THE 4 PARTICIPANTS
import re
import matplotlib.pyplot as plt

fig, axs = plt.subplots(2, 2, figsize=(10, 10))

adhd_participants = ['1', '8']
non_adhd_participants = ['32', '43']

for i, participant in enumerate(adhd_participants + non_adhd_participants):
    row = i // 2
    col = i % 2

    for j in range(min(1000, len(x_values['x' + participant]) - 1)):
        if j in saccades_fixations['participant' + participant]['fixations']:
            axs[row, col].plot(x_values['x' + participant][j], y_values['y' + participant][j], 'bo', markersize=5, label='Fixations')
        elif j in saccades_fixations['participant' + participant]['saccades']:
            axs[row, col].plot(x_values['x' + participant][j], y_values['y' + participant][j], 'ro', markersize=2, label='Saccades')

    axs[row, col].set_title(f'Participant {participant}')
    if col == 0:
        axs[row, col].set_ylabel('Y-Position')
    if row == 1:
        axs[row, col].set_xlabel('X-Position')

# Add titles for the top and bottom rows
fig.text(0.5, 0.92, "ADHD participants", fontsize=12, ha='center', va='bottom', fontweight='bold')
fig.text(0.5, 0.48, "non-ADHD participants", fontsize=12, ha='center', va='bottom', fontweight='bold')

# Increase the space between the rows
fig.subplots_adjust(hspace=0.4)

# Add a custom legend
custom_legend = [plt.Line2D([0], [0], color='b', marker='o', linestyle='', markersize=5, label='Fixations'),
                 plt.Line2D([0], [0], color='r', marker='o', linestyle='', markersize=2, label='Saccades')]
fig.legend(handles=custom_legend, loc='upper left', bbox_to_anchor=(0.8, 0.95))
#fig.legend(handles=custom_legend, loc='center right', bbox_to_anchor=(0.9, 0.95))

plt.savefig('scatter_plots.jpg', dpi=300, bbox_inches='tight')
plt.show()
